<a href="https://colab.research.google.com/github/ryabubaker/Network-Analysis-Actors-Collaboration/blob/main/API_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # to make TMDB API calls
import locale # to format currency as USD
locale.setlocale( locale.LC_ALL, '' )

import pandas as pd
#import matplotlib
import matplotlib.pyplot as plt
#from matplotlib.ticker import FuncFormatter # to format currency on charts axis

import itertools
from pandas import json_normalize 
import networkx as nx
import datetime


api_key = "f726183f78328796b0ee6de0e081480c" # get TMDB API key

In [ ]:
import json
import csv
import pandas as pd

# Read the JSON file in chunks, filtering each chunk for movies with a popularity value of 50 or more
def filter_chunk(chunk):
    return chunk[chunk['popularity'] >= 3]

df_chunk = pd.read_json("/content/movie_ids_04_05_2023.json", lines=True, chunksize=20000)
filtered_chunks = [filter_chunk(chunk) for chunk in df_chunk]

# Concatenate the filtered chunks into a single DataFrame
df = pd.concat(filtered_chunks)

# Select only the first 30,000 rows of the DataFrame
df = df.iloc[:50000]

# Select the "original_title" and "id" columns
df = df[["original_title", "id"]]

# Convert the DataFrame to a CSV file
df.to_csv("MoviesID.csv", index=False)

In [ ]:
movies = pd.read_csv("/content/MoviesID.csv")

In [ ]:
import concurrent.futures
import requests

def get_movie_data(movie_id, title):
    # Fetch movie details
    query_movie = "https://api.themoviedb.org/3/movie/" + str(movie_id) + "?api_key=" + api_key + "&language=en-US"
    response_movie = requests.get(query_movie)
    
    if response_movie.status_code == 200:
        movie_details = response_movie.json()
        year_of_release = movie_details['release_date'][:4]
        director = ""
        
        # Fetch movie credits
        query_credits = "https://api.themoviedb.org/3/movie/" + str(movie_id) + "/credits?api_key=" + api_key + "&language=en-US"
        response_credits = requests.get(query_credits)
        
        if response_credits.status_code == 200:
            movie_credits = response_credits.json()
            
            # Extract director information
            for crew_member in movie_credits['crew']:
                if crew_member['job'] == 'Director':
                    director = crew_member['name']
                    break
            
            # Extract cast information
            cast = [cast_member['name'] for cast_member in movie_credits['cast']]
            
            # Return the extracted information as a dictionary
            return {
                "id": movie_id,
                "title": title,
                "year_of_release": year_of_release,
                "director": director,
                "cast": cast
            }

with concurrent.futures.ThreadPoolExecutor() as executor:
    movie_data = list(executor.map(get_movie_data, movies['id'], movies['original_title']))


In [ ]:
# Filter out None values from the movie_data list
filtered_movie_data = [data for data in movie_data if data is not None]

# Create a DataFrame using the filtered movie data
movie_df = pd.DataFrame(filtered_movie_data)



In [ ]:
# Export the DataFrame to a CSV file
movie_df.to_csv('movie_data.csv', encoding='utf-8', index=False)